In [2]:
import os
import sys

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Inicializar a sessão Spark
spark = SparkSession.builder.appName("CustomGroupBy").getOrCreate()

# Carregar o DataFrame original
data = [
    (1, 1, 'Naruto'), 
    (1, 2, 'Anime2'), 
    (1, 3, 'Anime3'), 
    (1, 4, 'One Piece')
]
columns = ["user_id", "anime_id", "name"]
df = spark.createDataFrame(data, columns)


In [30]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *


@pandas_udf(ArrayType(
                StructType([
                    StructField('id', IntegerType()),
                    StructField('name', StringType()),
                    StructField('img', StringType())
])), PandasUDFType.GROUPED_AGG)  
def min_udf(anime_ids, anime_names):
    last_animes = []
    for anime_id, anime_name in zip(anime_ids[-3:], anime_names[-3:]):
        last_animes.append({
            'id': anime_id,
            'name': anime_name,
            'img': 'img-path'
        })
    return last_animes

result = df.groupby('user_id').agg(min_udf(df.anime_id, df.name)).collect()

print(result)



[Row(user_id=1, min_udf(anime_id, name)=[Row(id=2, name='Anime2', img='img-path'), Row(id=3, name='Anime3', img='img-path'), Row(id=4, name='One Piece', img='img-path')])]
